In [55]:
from lxml import etree, html
from lxml.html.clean import clean_html
import pandas as pd
from datetime import date, datetime
import requests
import re
import os
import shutil
from ASECORP_BBDD import tagea_BBDD_ASECORP, devuelve_patrones
from selenium import webdriver
import time

# Mueve todos los fichero del directorio de trabajo a otro de archivo

# Recoge resumen diario del BOE de hoy

In [56]:

today = date.today()

# dd/mm/YYYY
hoy = today.strftime("%Y%m%d")
print("Fecha de Hoy =", hoy)

# dd
d = today.strftime("%d")
print("dia =", d)

# mm
m = today.strftime("%m")
print("mes =", m)

# YYYY
Y = today.strftime("%Y")
print("año =", Y)

print(today.strftime("%d/%m/%Y"))

Fecha de Hoy = 20210309
dia = 09
mes = 03
año = 2021
09/03/2021


In [57]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

In [58]:
# Define expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
#pattern = ['Ley [0-9]+\/[0-9]+','Ley Orgánica [0-9]+\/[0-9]+','Decreto [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento \(UE\) [0-9]+\/[0-9]+', 'Reglamento de Ejeución \(UE\) [0-9]+\/[0-9]+' ,'Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Acuerdo Multilateral RID [0-9]+\/[0-9]+', 'Circular [0-9]+\/[0-9]+', 'Decisión \(UE\) [0-9]+\/[0-9]+', 'Decisión de Ejecución \(UE\) [0-9]+\/[0-9]+', 'Instrucción IS\-[0-9]+']
pattern = devuelve_patrones()

# ------------------------------------------------------------------------

# DOGC

In [59]:
URL_HTML_resumen = "https://dogc.gencat.cat/es/inici/"

In [60]:
### Necesita libreria Selenium para renderizar JS script

options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

driver.get(URL_HTML_resumen)
#print(driver.page_source)
response = driver.page_source

sumario_HTML = html.fromstring(response)
#print(sumario_HTML.xpath('//h2[@class="titol-component"]/a/@href'))
numero_diario = sumario_HTML.xpath('//h2[@class="titol-component"]/a/text()')
secciones = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/text()')
secciones_link = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/@href')
secciones_linkHTML = [ ('https://dogc.gencat.cat' + item) for item in secciones_link ]
# numero_pagina = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[3]/td[4]/text()')
# NID = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[2]/td[2]/text()')
# rango = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[5]/td[2]/text()')
# organo_emisor = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[7]/td[2]/text()')
# 
# print(numero_diario[0].strip(), numero_pagina[0].strip(), NID[0].strip(), rango[0].strip(), organo_emisor[0].strip())
# print()
print(numero_diario)
print(secciones)
print(secciones_linkHTML)

driver.quit()

['DOGC núm. 8360']
['Disposiciones', 'Cargos y personal', 'Anuncios de la Generalidad de Cataluña', 'Anuncios de la Administración local', 'Anuncios de la Administración de justicia ', 'Anuncios varios']
['https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8360#section-Disposiciones', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8360#section-Cargos_y_personal', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8360#section-Anuncios_de_la_Generalidad_de_Cataluña', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8360#section-Anuncios_de_la_Administración_local', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8360#section-Anuncios_de_la_Administración_de_justicia_', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8360#section-Anuncios_varios']


In [61]:
#response

## Recoge Nombre Secciones Sumario

In [62]:
secciones = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/text()')
#print(secciones)
df_secciones_sumarios = pd.DataFrame()
for seccion in secciones:
    df_secciones_sumarios = df_secciones_sumarios.append({'Seccion': seccion}, ignore_index=True)


In [63]:
df_secciones_sumarios

,Seccion
0,Disposiciones
1,Cargos y personal
2,Anuncios de la Generalidad de Cataluña
3,Anuncios de la Administración local
4,Anuncios de la Administración de justicia
5,Anuncios varios


## Recoge Valores para formar URLs Secciones Sumario

In [64]:
URL_base_sumario = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/@href')
URL_sumario = [ ('https://dogc.gencat.cat' + item) for item in secciones_link ]

df_URL_sumarios = pd.DataFrame()

for URL_seccion in URL_base_sumario:    
    URL_sumario = 'https://dogc.gencat.cat' + str(URL_seccion)
    print(URL_sumario)
    df_URL_sumarios = df_URL_sumarios.append({'URL_Seccion': URL_sumario}, ignore_index=True)

https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8360#section-Disposiciones
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8360#section-Cargos_y_personal
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8360#section-Anuncios_de_la_Generalidad_de_Cataluña
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8360#section-Anuncios_de_la_Administración_local
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8360#section-Anuncios_de_la_Administración_de_justicia_
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8360#section-Anuncios_varios


In [65]:
df_URL_sumarios

,URL_Seccion
0,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
1,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
2,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
3,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
4,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
5,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...


In [66]:
 df_sumarios = pd.concat([df_secciones_sumarios, df_URL_sumarios], axis=1)

In [67]:
df_sumarios

,Seccion,URL_Seccion
0,Disposiciones,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
1,Cargos y personal,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
2,Anuncios de la Generalidad de Cataluña,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
3,Anuncios de la Administración local,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
4,Anuncios de la Administración de justicia,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
5,Anuncios varios,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...


In [68]:
df_sumarios['URL_Seccion'][0]

'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8360#section-Disposiciones'

## Recoge Items en Seccion Disposiciones

In [69]:
URL_HTML_disposiciones = df_sumarios[df_sumarios['Seccion'] == 'Disposiciones']

In [70]:
print(URL_HTML_disposiciones['Seccion'][0])

Disposiciones


In [71]:
print(URL_HTML_disposiciones['URL_Seccion'][0])

https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8360#section-Disposiciones


In [72]:
URL_HTML_disposiciones = URL_HTML_disposiciones['URL_Seccion'][0]

In [73]:
URL_HTML_disposiciones

'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8360#section-Disposiciones'

In [74]:
options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

driver.get(URL_HTML_disposiciones)
#print(driver.page_source)
response = driver.page_source
### Recoge Items en Seccion Disposiciones
disposiciones_HTML = html.fromstring(response)

#print(html.tostring(disposiciones_HTML))
#print('____________________')

departamentos = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/h3/text()') # Sólo Sección disposiciones
disposiciones = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/ul/descendant::li/div/a/text()')
disposiciones_link = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/ul/descendant::li/div/a/@href')
disposiciones_pdf = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/ul/descendant::li/div/div/a/@href')

# for i in range(len(departamentos)):
#     print(departamentos[i])
#     i += 1
#     #print(disposiciones_HTML.xpath('//*[@class="wrapper-disposicions"]/ul[' + str(i) + ']//li/div/a/text()'))
#     print(disposiciones_HTML.xpath('//*[@class="wrapper-disposicions"]/ul[1]//li/div/a/text()'))

print(departamentos)
print(disposiciones)
print(disposiciones_link)
print(disposiciones_pdf)

driver.quit()


['Departamento de la Presidencia', 'Departamento de Salud', 'Departamento de Territorio y Sostenibilidad', 'Departamento de Cultura', 'Departamento de Empresa y Conocimiento', 'Departamento de Agricultura, Ganadería, Pesca y Alimentación']
['RESOLUCIÓN PRE/613/2021, de 4 de marzo, por la que se abre la convocatoria para la concesión de ayudas extraordinarias y de emergencia para hacer frente a las afectaciones económicas y sociales producidas en el sector de la actividad física y el deporte a causa de la suspensión y la limitación de la apertura al público de las instalaciones deportivas en Cataluña atendiendo la publicación de las resoluciones SLT/1/2021, de 4 de enero, SLT/67/2021, de 17 de enero, y SLT/133/2021, de 22 de enero, como consecuencia de la crisis sanitaria, económica y social de la COVID-19 (ref. BDNS 551891).', 'RESOLUCIÓN SLT/622/2021, de 5 de febrero, por la que se abre la convocatoria para el año 2021, para la concesión de subvenciones para la financiación a entidade

In [75]:
len(departamentos)

6

In [76]:
DOGC_sumarios = pd.DataFrame(columns=['item_Title','item_urlHTML','item_urlPDF'])

In [77]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF


In [78]:
lista = []
for disposicion in disposiciones:
    lista.append(disposicion)

DOGC_sumarios['item_Title'] = lista

lista = []   
for link_HTML in disposiciones_link:
    #print(link)
    lista.append('https://dogc.gencat.cat' + str(link_HTML))

DOGC_sumarios['item_urlHTML'] = lista

lista = []   
for link_PDF in disposiciones_pdf:
    #print(link)
    lista.append(link_PDF)

DOGC_sumarios['item_urlPDF'] = lista

In [79]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF
0,"RESOLUCIÓN PRE/613/2021, de 4 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
1,"RESOLUCIÓN SLT/622/2021, de 5 de febrero, por ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
2,"EDICTO de 2 de marzo de 2021, sobre un acuerdo...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
3,"EDICTO de 2 de marzo de 2021, sobre un acuerdo...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
4,"RESOLUCIÓN CLT/616/2021, de 4 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
5,"RESOLUCIÓN CLT/617/2021, de 4 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
6,"RESOLUCIÓN CLT/618/2021, de 4 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
7,"RESOLUCIÓN EMC/621/2021, de 24 de febrero, por...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
8,"RESOLUCIÓN EMC/604/2021, de 2 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
9,"ORDEN ARP/54/2021, de 3 de marzo, por la que s...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...


In [80]:
DOGC_sumarios['item_urlHTML'][1]

'https://dogc.gencat.cat/es/document-del-dogc/?documentId=895119'

In [81]:
### Recoge información de página de detalle con Selenium
### es necesario ya que la página se genera con un JS y
### hay que renderizarla con un headless web browser

options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

DOGC_sumarios['item_fullText'] = ''
DOGC_sumarios['item_urlXML'] = ''
DOGC_sumarios['tipo_doc'] = ''
DOGC_sumarios['fecha_doc'] = ''
DOGC_sumarios['numero_doc'] = ''
DOGC_sumarios['numero_control'] = ''
DOGC_sumarios['entidad_emisora'] = ''
DOGC_sumarios['CVE'] = ''
DOGC_sumarios['numero_DOGC'] = ''
DOGC_sumarios['fecha_DOGC'] = ''
DOGC_sumarios['seccion_DOGC'] = ''

for i, row in DOGC_sumarios.iterrows():
    # carga página HTML y genera árbol
    driver.get(row['item_urlHTML'])
    #print(driver.page_source)

    # clica sobre desplegable 'datos del documento' para generar contenido
    driver.find_element_by_xpath('//div[@class="panel-title"]/a[@href="#disposicions"]').click()
    time.sleep(3)

    response = driver.page_source
    sumario_HTML = html.fromstring(response)

    # Genera columnas de la tabla con Datos de los documentos
    item_urlXML = sumario_HTML.xpath('//*[@id="download"]/a[4]/@href')

    tipo_doc = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Tipo de documento")]/text()[2]')
    fecha_doc = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Fecha del documento")]/text()[2]')
    numero_doc = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Número de documento")]/text()[2]')
    numero_control = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Número de control")]/text()[2]')
    entidad_emisora = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Entidad emisora")]/text()[2]')
    CVE = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"CVE")]/text()[2]')
    numero_DOGC = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Número del DOGC")]/text()[2]')
    fecha_DOGC = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Fecha del DOGC")]/text()[2]')
    seccion_DOGC = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Sección del DOGC")]/text()[2]')

    DOGC_sumarios['item_urlXML'][i] = item_urlXML

    DOGC_sumarios['tipo_doc'][i] = tipo_doc
    DOGC_sumarios['fecha_doc'][i] = fecha_doc
    DOGC_sumarios['numero_doc'][i] = numero_doc
    DOGC_sumarios['numero_control'][i] = numero_control
    DOGC_sumarios['entidad_emisora'][i] = entidad_emisora
    DOGC_sumarios['CVE'][i] = CVE
    DOGC_sumarios['numero_DOGC'][i] = numero_DOGC
    DOGC_sumarios['fecha_DOGC'][i] = fecha_DOGC
    DOGC_sumarios['seccion_DOGC'][i] = seccion_DOGC

    # Recoge en una lista todos los parágrafos del texto completo
    fullText = []
    for div in sumario_HTML.xpath('//*[@id="fullText"]/div/div/div'):
        fullText.append(div.text_content())

    # Convierte la lista en una string
    fullText = ' '.join(fullText)

    DOGC_sumarios['item_fullText'][i] = fullText


driver.quit()

In [82]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF,item_fullText,item_urlXML,tipo_doc,fecha_doc,numero_doc,numero_control,entidad_emisora,CVE,numero_DOGC,fecha_DOGC,seccion_DOGC
0,"RESOLUCIÓN PRE/613/2021, de 4 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"Mediante la Resolución SLT/1/2021, de 4 de ene...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[04/03/2021],[PRE/0613/2021],[21064052],[Consejo Catalán del Deporte],[CVE-DOGC-B-21064052-2021],[8360],[09/03/2021],[Disposiciones]
1,"RESOLUCIÓN SLT/622/2021, de 5 de febrero, por ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"El Departamento de Salud, mediante la concesió...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[05/02/2021],[SLT/0622/2021],[21064039],[Departamento de Salud],[CVE-DOGC-B-21064039-2021],[8360],[09/03/2021],[Disposiciones]
2,"EDICTO de 2 de marzo de 2021, sobre un acuerdo...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,\nLa Comisión Territorial de Urbanismo de Gir...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Edicto ],[02/03/2021],[],[21061051],[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21061051-2021],[8360],[09/03/2021],[Disposiciones]
3,"EDICTO de 2 de marzo de 2021, sobre un acuerdo...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,La Comisión Territorial de Urbanismo de Girona...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Edicto ],[02/03/2021],[],[21061037],[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21061037-2021],[8360],[09/03/2021],[Disposiciones]
4,"RESOLUCIÓN CLT/616/2021, de 4 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,Visto el Acuerdo del Consejo de Administración...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[04/03/2021],[CLT/0616/2021],[21064041],[Instituto Catalán de las Empresas Culturales],[CVE-DOGC-B-21064041-2021],[8360],[09/03/2021],[Disposiciones]
5,"RESOLUCIÓN CLT/617/2021, de 4 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,Visto el Acuerdo del Consejo de Administración...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[04/03/2021],[CLT/0617/2021],[21064042],[Instituto Catalán de las Empresas Culturales],[CVE-DOGC-B-21064042-2021],[8360],[09/03/2021],[Disposiciones]
6,"RESOLUCIÓN CLT/618/2021, de 4 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,Visto el Acuerdo del Consejo de Administración...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[04/03/2021],[CLT/0618/2021],[21064043],[Instituto Catalán de las Empresas Culturales],[CVE-DOGC-B-21064043-2021],[8360],[09/03/2021],[Disposiciones]
7,"RESOLUCIÓN EMC/621/2021, de 24 de febrero, por...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"La Resolución EMC/283/2020, de 7 de febrero (D...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[24/02/2021],[EMC/0621/2021],[21064046],[Agencia para la Calidad del Sistema Universit...,[CVE-DOGC-B-21064046-2021],[8360],[09/03/2021],[Disposiciones]
8,"RESOLUCIÓN EMC/604/2021, de 2 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,El Consejo de Administración de 30 de abril de...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[02/03/2021],[EMC/0604/2021],[21063053],[Agencia para la Competitividad de la Empresa],[CVE-DOGC-B-21063053-2021],[8360],[09/03/2021],[Disposiciones]
9,"ORDEN ARP/54/2021, de 3 de marzo, por la que s...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJ

In [83]:
DOGC_sumarios['item_fullText'][0]

'Mediante la Resolución SLT/1/2021, de 4 de enero, se prorrogan y se modifican las medidas en materia de salud pública para la contención del brote epidémico de la pandemia de COVID-19 en el territorio de Cataluña, y se adoptan una serie de nuevas medidas de salud pública para la contención del brote epidémico de la pandemia de la COVID-19 en el territorio de Cataluña, entre las 0.00 h del día 7 hasta las 0.00 h del día 18 de enero. La Resolución SLT/67/2021, de 17 de enero, y la Resolución SLT/133/2021, de 22 de enero, prorrogan de nuevo las medidas de suspensión y limitación antes señaladas, durante 3 semanas más.\n Estas nuevas medidas han implicado la suspensión y la limitación de la apertura al público de las instalaciones y de los equipamientos deportivos, a excepción de los centros de tecnificación y rendimiento deportivos, tanto de titularidad pública como privada, así como de las instalaciones y equipamientos deportivos que tengan que acoger entrenamientos y competición profes

In [84]:
# Crea nueva columna vacía de tipo lista en tabla_analisis
#DOCM_sumarios['Referencias_completas'] = [[] for i in range(len(tabla_analisis))]
DOGC_sumarios['Tags'] = [[] for i in range(len(DOGC_sumarios))]
DOGC_sumarios['Match_ASECORP_BBDD'] = [[] for i in range(len(DOGC_sumarios))]

In [85]:
pattern = ['Ley [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento \(UE\) [0-9]+\/[0-9]+', 'Reglamento de Ejeución \(UE\) [0-9]+\/[0-9]+' ,'Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Circular [0-9]+\/[0-9]+', 'Decisión \(UE\) [0-9]+\/[0-9]+', 'Decisión de Ejecución \(UE\) [0-9]+\/[0-9]+']

In [86]:
for i, row in DOGC_sumarios.iterrows():
    # Busca expresiones REGX coincidentes con Patrones definidos
    DOGC_sumarios['Tags'][i] = re.findall('|'.join(pattern), str(row['item_fullText']), flags=re.IGNORECASE)
    print(DOGC_sumarios['Tags'][i])

['Resolución SLT/1/2021', 'Resolución SLT/67/2021', 'Resolución SLT/133/2021', 'Ley 43/2020', 'Resolución PRE/392/2021', 'Ley 4/2020', 'Resolución PRE/392/2021', 'Resolución PRE/392/2021', 'Resolución PRE/392/2021', 'Ley 38/2003', 'Orden GRI/233/2015', 'Orden PRE/5/2017', 'Ley 26/2010', 'Ley 39/2015', 'Ley 26/2010', 'Ley 26/2010', 'Ley 39/2015']
['Orden SLT/257/2017', 'Ley 19/2014', 'Ley 38/2003', 'Real decreto 887/2006', 'Ley 4/2020', 'Orden SLT/257/2017', 'Orden SLT/257/2017', 'Orden SLT/198/2018', 'Orden SLT/257/2017', 'Orden SLT/257/2017', 'Ley 38/2003', 'Ley 26/2010', 'Orden SLT/257/2017', 'Orden SLT/257/2017', 'Ley 39/2015']
['Ley 3/2012', 'Ley 39/2015', 'Ley 29/1998', 'Ley 3/2012', 'Ley 2/2007', 'Ley 3/2012']
['Ley 39/2015', 'Ley 29/1998', 'Ley 3/2012', 'Ley 2/2007', 'Ley 3/2012']
['Resolución CLT/3142/2020', 'Resolución CLT/187/2021', 'Ley 20/2010', 'Ley 38/2003', 'Ley 20/2010', 'Resolución CLT/1173/2020', 'Ley 17/2015']
['Resolución CLT/3142/2020', 'Resolución CLT/187/2021', '

In [87]:
# Elimina Tags duplicados
for i, row in DOGC_sumarios.iterrows():
    DOGC_sumarios['Tags'][i] = list(set(DOGC_sumarios['Tags'][i]))
    print(DOGC_sumarios['Tags'][i])

['Ley 39/2015', 'Resolución SLT/67/2021', 'Orden GRI/233/2015', 'Ley 38/2003', 'Ley 43/2020', 'Orden PRE/5/2017', 'Resolución SLT/1/2021', 'Resolución SLT/133/2021', 'Resolución PRE/392/2021', 'Ley 4/2020', 'Ley 26/2010']
['Ley 39/2015', 'Real decreto 887/2006', 'Orden SLT/198/2018', 'Ley 38/2003', 'Ley 19/2014', 'Orden SLT/257/2017', 'Ley 4/2020', 'Ley 26/2010']
['Ley 39/2015', 'Ley 2/2007', 'Ley 3/2012', 'Ley 29/1998']
['Ley 39/2015', 'Ley 29/1998', 'Ley 2/2007', 'Ley 3/2012']
['Ley 38/2003', 'Resolución CLT/1173/2020', 'Resolución CLT/187/2021', 'Resolución CLT/3142/2020', 'Ley 17/2015', 'Ley 20/2010']
['Ley 18/1982', 'Ley 38/2003', 'Resolución CLT/807/2019', 'Resolución CLT/187/2021', 'Resolución CLT/3142/2020', 'Resolución CLT/298/2020']
['Resolución CLT/1356/2017', 'Ley 38/2003', 'Real decreto 463/2020', 'Resolución CLT/187/2021', 'Resolución CLT/3142/2020', 'Ley 17/2015']
['Ley 39/2015', 'Resolución EMC/460/2020', 'Resolución EMC/283/2020', 'Ley 38/2003', 'Ley 29/1998', 'Resoluc

In [88]:
regex_result = []
[ regex_result.append(tag) for tags in DOGC_sumarios['Tags'] for tag in tags ]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [89]:
print(regex_result)

['Ley 39/2015', 'Resolución SLT/67/2021', 'Orden GRI/233/2015', 'Ley 38/2003', 'Ley 43/2020', 'Orden PRE/5/2017', 'Resolución SLT/1/2021', 'Resolución SLT/133/2021', 'Resolución PRE/392/2021', 'Ley 4/2020', 'Ley 26/2010', 'Ley 39/2015', 'Real decreto 887/2006', 'Orden SLT/198/2018', 'Ley 38/2003', 'Ley 19/2014', 'Orden SLT/257/2017', 'Ley 4/2020', 'Ley 26/2010', 'Ley 39/2015', 'Ley 2/2007', 'Ley 3/2012', 'Ley 29/1998', 'Ley 39/2015', 'Ley 29/1998', 'Ley 2/2007', 'Ley 3/2012', 'Ley 38/2003', 'Resolución CLT/1173/2020', 'Resolución CLT/187/2021', 'Resolución CLT/3142/2020', 'Ley 17/2015', 'Ley 20/2010', 'Ley 18/1982', 'Ley 38/2003', 'Resolución CLT/807/2019', 'Resolución CLT/187/2021', 'Resolución CLT/3142/2020', 'Resolución CLT/298/2020', 'Resolución CLT/1356/2017', 'Ley 38/2003', 'Real decreto 463/2020', 'Resolución CLT/187/2021', 'Resolución CLT/3142/2020', 'Ley 17/2015', 'Ley 39/2015', 'Resolución EMC/460/2020', 'Resolución EMC/283/2020', 'Ley 38/2003', 'Ley 29/1998', 'Resolución UNI

In [90]:
## Elimina duplicados
boletin_flat_list = list(set(regex_result))

In [91]:
boletin_flat_list

['Resolución SLT/67/2021',
 'Ley 18/1982',
 'Orden SLT/198/2018',
 'Resolución SLT/133/2021',
 'Resolución CLT/187/2021',
 'Resolución EMC/1446/2020',
 'Orden AAM/387/2014',
 'Reglamento (UE) 1305/2013',
 'Reglamento (UE) 809/2014',
 'Orden SLT/257/2017',
 'Ley 17/2015',
 'Resolución CLT/298/2020',
 'Ley 7/2012',
 'Ley 4/2020',
 'Reglamento (UE) 1306/2013',
 'Ley 39/2015',
 'Ley 4/2008',
 'Resolución EMC/460/2020',
 'Ley 43/2020',
 'Resolución EMC/3294/2020',
 'Resolución CLT/807/2019',
 'Ley 59/2003',
 'Ley 29/1998',
 'Reglamento (UE) 1303/2013',
 'Ley 19/2014',
 'Real decreto legislativo 9/2017',
 'Real Decreto 887/2006',
 'Real decreto 1335/2011',
 'Resolución EMC/283/2020',
 'Orden PRE/5/2017',
 'Resolución SLT/1/2021',
 'Resolución PRE/392/2021',
 'Reglamento (UE) 808/2014',
 'Reglamento (UE) 2016/679',
 'Resolución UNI/73/2003',
 'Ley 5/2011',
 'Ley 26/2010',
 'Reglamento (UE) 2020/2220',
 'Real decreto 887/2006',
 'Resolución CLT/1356/2017',
 'Orden GRI/233/2015',
 'Ley 38/2003'

In [92]:
# ## Importa BBDD ASECORP

# Inicializa datos de BBDD_ASECORP
boletin_ASECORP_flat_list = []
ASECORP_BBDD = pd.DataFrame()
ambitos = []

# Incluir en llamada a función el ambito territorial como lista
# si no se especifica ámbito los incluye todos
boletin_ASECORP_flat_list, ASECORP_BBDD, ambitos = tagea_BBDD_ASECORP(['España','Europa','Cataluña'])

print(ambitos)


['España', 'Europa', 'Cataluña']


In [93]:
#boletin_ASECORP_flat_list

In [94]:
## Busca coincidencias entre lista boletines BOEs explorados y lista boletines de BBDD ASECORP
set(boletin_flat_list) & set(boletin_ASECORP_flat_list)

set()

In [95]:
#DOCM_sumarios['Tags'].isin(ASECORP_BBDD_BOE['Tags'])
for row_to_compare in DOGC_sumarios['Tags']:
    for row_comparing in ASECORP_BBDD['Tags']:
        if set(row_comparing) & set(row_to_compare):
            print(set(row_comparing) & set(row_to_compare))

{'Resolución SLT/1/2021'}
{'Resolución SLT/133/2021'}
{'Resolución SLT/67/2021'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 17/2015'}
{'Ley 17/2015'}
{'Real decreto 463/2020'}
{'Real decreto 463/2020'}


In [96]:
for i, row_to_compare in DOGC_sumarios.iterrows():
    for j, row_comparing in ASECORP_BBDD.iterrows():
        if set(row_to_compare['Tags']) & set(row_comparing['Tags']):
            DOGC_sumarios['Match_ASECORP_BBDD'][i].append (ASECORP_BBDD['Codigo'][j])
            print(str(set(row_to_compare['Tags']) & set(row_comparing['Tags'])) + ' ' + str(row_comparing['Codigo']))

{'Resolución SLT/1/2021'} RES SLT/1/2021 CAT
{'Resolución SLT/133/2021'} RES SLT/133/2021 CAT
{'Resolución SLT/67/2021'} RES SLT/67/2021 CAT
{'Ley 3/2012'} DLE 3/2012 CAT
{'Ley 3/2012'} LEY 14/2015 CAT
{'Ley 3/2012'} LEY 3/2012 CAT
{'Ley 3/2012'} LEY 3/2012 ESP
{'Ley 3/2012'} DLE 3/2012 CAT
{'Ley 3/2012'} LEY 14/2015 CAT
{'Ley 3/2012'} LEY 3/2012 CAT
{'Ley 3/2012'} LEY 3/2012 ESP
{'Ley 17/2015'} LEY 17/2015 ESP
{'Ley 17/2015'} LEY 17/2015 ESP
{'Real decreto 463/2020'} RES TES/1034/2020 CAT
{'Real decreto 463/2020'} RES TES/973/2020 CAT


In [97]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF,item_fullText,item_urlXML,tipo_doc,fecha_doc,numero_doc,numero_control,entidad_emisora,CVE,numero_DOGC,fecha_DOGC,seccion_DOGC,Tags,Match_ASECORP_BBDD
0,"RESOLUCIÓN PRE/613/2021, de 4 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"Mediante la Resolución SLT/1/2021, de 4 de ene...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[04/03/2021],[PRE/0613/2021],[21064052],[Consejo Catalán del Deporte],[CVE-DOGC-B-21064052-2021],[8360],[09/03/2021],[Disposiciones],"[Ley 39/2015, Resolución SLT/67/2021, Orden GR...","[RES SLT/1/2021 CAT, RES SLT/133/2021 CAT, RES..."
1,"RESOLUCIÓN SLT/622/2021, de 5 de febrero, por ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"El Departamento de Salud, mediante la concesió...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[05/02/2021],[SLT/0622/2021],[21064039],[Departamento de Salud],[CVE-DOGC-B-21064039-2021],[8360],[09/03/2021],[Disposiciones],"[Ley 39/2015, Real decreto 887/2006, Orden SLT...",[]
2,"EDICTO de 2 de marzo de 2021, sobre un acuerdo...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,\nLa Comisión Territorial de Urbanismo de Gir...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Edicto ],[02/03/2021],[],[21061051],[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21061051-2021],[8360],[09/03/2021],[Disposiciones],"[Ley 39/2015, Ley 2/2007, Ley 3/2012, Ley 29/1...","[DLE 3/2012 CAT, LEY 14/2015 CAT, LEY 3/2012 C..."
3,"EDICTO de 2 de marzo de 2021, sobre un acuerdo...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,La Comisión Territorial de Urbanismo de Girona...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Edicto ],[02/03/2021],[],[21061037],[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21061037-2021],[8360],[09/03/2021],[Disposiciones],"[Ley 39/2015, Ley 29/1998, Ley 2/2007, Ley 3/2...","[DLE 3/2012 CAT, LEY 14/2015 CAT, LEY 3/2012 C..."
4,"RESOLUCIÓN CLT/616/2021, de 4 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,Visto el Acuerdo del Consejo de Administración...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[04/03/2021],[CLT/0616/2021],[21064041],[Instituto Catalán de las Empresas Culturales],[CVE-DOGC-B-21064041-2021],[8360],[09/03/2021],[Disposiciones],"[Ley 38/2003, Resolución CLT/1173/2020, Resolu...",[LEY 17/2015 ESP]
5,"RESOLUCIÓN CLT/617/2021, de 4 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,Visto el Acuerdo del Consejo de Administración...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[04/03/2021],[CLT/0617/2021],[21064042],[Instituto Catalán de las Empresas Culturales],[CVE-DOGC-B-21064042-2021],[8360],[09/03/2021],[Disposiciones],"[Ley 18/1982, Ley 38/2003, Resolución CLT/807/...",[]
6,"RESOLUCIÓN CLT/618/2021, de 4 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,Visto el Acuerdo del Consejo de Administración...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[04/03/2021],[CLT/0618/2021],[21064043],[Instituto Catalán de las Empresas Culturales],[CVE-DOGC-B-21064043-2021],[8360],[09/03/2021],[Disposiciones],"[Resolución CLT/1356/2017, Ley 38/2003, Real d...","[LEY 17/2015 ESP, RES TES/1034/2020 CAT, RES T..."
7,"RESOLUCIÓN EMC/621/2021, de 24 de febrero, por...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"La Resolución EMC/283/2020, de 7 de febrero (D...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[24/02/2021],[EMC/0621/2021],[21064046],[Agencia para la Calidad del Sistema Universit...,[CVE-DOGC

# Genera Fichero EXCEL de resultados

In [98]:
DOGC_sumarios_final = DOGC_sumarios

In [99]:
DOGC_sumarios_final.dtypes

item_Title            object
item_urlHTML          object
item_urlPDF           object
item_fullText         object
item_urlXML           object
tipo_doc              object
fecha_doc             object
numero_doc            object
numero_control        object
entidad_emisora       object
CVE                   object
numero_DOGC           object
fecha_DOGC            object
seccion_DOGC          object
Tags                  object
Match_ASECORP_BBDD    object
dtype: object

In [100]:
## Cambia orden de columnas y elimina las no necesarias  
DOGC_sumarios_final.rename(columns={'CVE': 'Item_id', 'item_Title': 'Item_Title', 'item_urlPDF': 'PDF_Link', 'fecha_DOGC': 'Fecha_publicacion'}, inplace=True)
DOGC_sumarios_final = DOGC_sumarios[['Item_id','Item_Title','PDF_Link','Fecha_publicacion','Tags','Match_ASECORP_BBDD']]

In [101]:
## Crea función que convierte lista a string en todas las columnas de DOCM_sumarios
## para evitar en presentación final los caracteres [' '] propios de las listas
def list2Str(lst):
    if type(lst) is list: # apply conversion to list columns
        return", ".join(lst)
    else:
        return lst

In [102]:

DOGC_sumarios_final = DOGC_sumarios_final.apply(lambda x: [list2Str(i) for i in x])

In [103]:
## Generar hyperlink a artículo BOE en CSV "=HYPERLINK("https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf";"BOE-A-2021-1474")"
## https://www.boe.es/diario_boe/xml.php?id=## https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf

DOGC_sumarios_final_CSV = DOGC_sumarios_final

In [104]:
DOGC_sumarios_final_CSV['item_id'] = '=HIPERVINCULO(' + '"' + DOGC_sumarios_final_CSV['PDF_Link'] + '";' \
                                                        + '"' + DOGC_sumarios_final_CSV['Item_id'] + '")'      

In [105]:
DOGC_sumarios_final_CSV['item_id'][0]

'=HIPERVINCULO("https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=895198&type=01&language=es_ES";"CVE-DOGC-B-21064052-2021")'

In [106]:
# Elimina columna PDF_Link
DOGC_sumarios_final_CSV = DOGC_sumarios_final_CSV[['Item_id','Item_Title','Fecha_publicacion','Tags','Match_ASECORP_BBDD']]

In [107]:
DOGC_sumarios_final_CSV

,Item_id,Item_Title,Fecha_publicacion,Tags,Match_ASECORP_BBDD
0,CVE-DOGC-B-21064052-2021,"RESOLUCIÓN PRE/613/2021, de 4 de marzo, por la...",09/03/2021,"Ley 39/2015, Resolución SLT/67/2021, Orden GRI...","RES SLT/1/2021 CAT, RES SLT/133/2021 CAT, RES ..."
1,CVE-DOGC-B-21064039-2021,"RESOLUCIÓN SLT/622/2021, de 5 de febrero, por ...",09/03/2021,"Ley 39/2015, Real decreto 887/2006, Orden SLT/...",
2,CVE-DOGC-B-21061051-2021,"EDICTO de 2 de marzo de 2021, sobre un acuerdo...",09/03/2021,"Ley 39/2015, Ley 2/2007, Ley 3/2012, Ley 29/1998","DLE 3/2012 CAT, LEY 14/2015 CAT, LEY 3/2012 CA..."
3,CVE-DOGC-B-21061037-2021,"EDICTO de 2 de marzo de 2021, sobre un acuerdo...",09/03/2021,"Ley 39/2015, Ley 29/1998, Ley 2/2007, Ley 3/2012","DLE 3/2012 CAT, LEY 14/2015 CAT, LEY 3/2012 CA..."
4,CVE-DOGC-B-21064041-2021,"RESOLUCIÓN CLT/616/2021, de 4 de marzo, por la...",09/03/2021,"Ley 38/2003, Resolución CLT/1173/2020, Resoluc...",LEY 17/2015 ESP
5,CVE-DOGC-B-21064042-2021,"RESOLUCIÓN CLT/617/2021, de 4 de marzo, por la...",09/03/2021,"Ley 18/1982, Ley 38/2003, Resolución CLT/807/2...",
6,CVE-DOGC-B-21064043-2021,"RESOLUCIÓN CLT/618/2021, de 4 de marzo, por la...",09/03/2021,"Resolución CLT/1356/2017, Ley 38/2003, Real de...","LEY 17/2015 ESP, RES TES/1034/2020 CAT, RES TE..."
7,CVE-DOGC-B-21064046-2021,"RESOLUCIÓN EMC/621/2021, de 24 de febrero, por...",09/03/2021,"Ley 39/2015, Resolución EMC/460/2020, Resoluci...",
8,CVE-DOGC-B-21063053-2021,"RESOLUCIÓN EMC/604/2021, de 2 de marzo, por la...",09/03/2021,"Ley 39/2015, Real decreto 887/2006, Ley 38/200...",
9,CVE-DOGC-B-21064047-2021,"ORDEN ARP/54/2021, de 3 de marzo, por la que s...",09/03/2021,"Ley 4/2008, Reglamento (UE) 2020/2220, Ley 39/...",


In [108]:
DOGC_sumarios_final_CSV.to_csv("./ASECORP/Resultados_Matching_DOGC_" + today.strftime("%Y%m%d") + ".csv" ,index=False) 